<div class="alert alert-block alert-warning">
    <span style='font-family:Georgia'>
        <b>Strategy for data cleaning: </b> 
        <ul>
            <li>create new column called total_score which is the total score of all subjects for each student</li>
            <li>rename the columns names</li>
            <li>remove Unnamed column from both data sets</li>
            <li>EthnicGroup in expanded data set, We will distribute the students who are not in a group to the groups that are smaller in number, and so on. </li>
            <li>ParentEduc: i think, when the paren eduction degree increaced the degree of the childeren will be increaced, so that the total score of child will 
            be important factor in detemining the paren eduction</li>
            <li>TestPrep: there is nothing has factor in detemining it so i will replace them by no_detected</li>
            <li>ParentMaritalStatus: there is nothing has factor in detemining it so i will replace them by no_detected</li>
            <li>PracticeSport: there is nothing has factor in detemining it so i will replace them by no_detected</li>
            <li>IsFirstChild: in my opinin if the total score was large, that means he is the first child, Due to his parents' interest in him because he is the first child</li>
            <li>NrSiblings: acctuly i don't have any idea for it so i think i will replace it by mean</li>
            <li>TransportMeans: will replaced by school_bus</li>
            <li>WklyStudyHours: it make sence that if the student has large score that he studys more houers so the score has factor in detemining it</li>
        </ul>       
    </span>    
</div>

In [53]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import collections 
pd.options.display.max_columns = None
plt.style.use('classic')
expanded_df = pd.read_csv(r'/home/ahmed/Ai/Analysis/data analysis projects/Students Exam Scores/archive/Expanded_data_with_more_features.csv')
orignal_df = pd.read_csv(r'/home/ahmed/Ai/Analysis/data analysis projects/Students Exam Scores/archive/Original_data_with_more_rows.csv')

In [54]:
temp_expanded_df = expanded_df.copy()

In [55]:
temp_expanded_df

,Unnamed: 0,Gender,EthnicGroup,ParentEduc,LunchType,TestPrep,ParentMaritalStatus,PracticeSport,IsFirstChild,NrSiblings,TransportMeans,WklyStudyHours,MathScore,ReadingScore,WritingScore
0,0,female,NaN,bachelor's degree,standard,none,married,regularly,yes,3.0,school_bus,< 5,71,71,74
1,1,female,group C,some college,standard,NaN,married,sometimes,yes,0.0,NaN,5 - 10,69,90,88
2,2,female,group B,master's degree,standard,none,single,sometimes,yes,4.0,school_bus,< 5,87,93,91
3,3,male,group A,associate's degree,free/reduced,none,married,never,no,1.0,NaN,5 - 10,45,56,42
4,4,male,group C,some college,standard,none,married,sometimes,yes,0.0,school_bus,5 - 10,76,78,75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30636,816,female,group D,high school,standard,none,single,sometimes,no,2.0,school_bus,5 - 10,59,61,65
30637,890,male,group E,high school,standard,none,single,regularly,no,1.0,private,5 - 10,58,53,51
30638,911,female,NaN,high school,free/reduced,completed,married,sometimes,no,1.0,private,5 - 10,61,70,67
30639,934,female,group D,associate's degree,standard,completed,married,regularly,no,3.0,school_bus,5 - 10,82,90,93


<a id="intro"></a>
<h2>   
      <font color = blue >
            <span style='font-family:Georgia'>
            creating class contains all functions i will use:
            </span>   
        </font>    
</h2>

In [56]:
class Data_Cleaning:
    
    # create a initial fuction which has self that assigned to data set
    def __init__(self) -> None:
        self.df = expanded_df.copy()
        return


    # here we will remove the column Unnamed: 0
    def remove(self):
        self.df.drop(columns=['Unnamed: 0'], inplace=True)
        return self.df
    


    # in this fucntion we will rename the columns
    def rename_columns(self) -> pd.DataFrame:
        
        # list contains old names
        columns = ['Gender', 'EthnicGroup', 'ParentEduc', 'LunchType', 'TestPrep', 'ParentMaritalStatus', 'PracticeSport', 'IsFirstChild',
                   'NrSiblings', 'TransportMeans', 'WklyStudyHours', 'MathScore', 'ReadingScore', 'WritingScore']

        # list contains new names
        new_name = ['gender', 'group', 'parent_education', 'lunch_type', 'test_preparation', 'parent_marital_status', 'practice_sport', 'first_child'
                    ,'number_of_siblings', 'transportation', 'week_study_hours', 'math_score', 'reading_score', 'writing_score']

        # there, i will put two lists in a dictionary to rename columns with it 
        mappe = {}
        for i, j in zip(columns, new_name):
            mappe[i] = j
  

        self.df.rename(columns=mappe, inplace=True)
        return self.df


    # function to create a new column that contains total_score  of each student
    def create_total_score(self):

        self.df['total_score'] = self.df['math_score'] + self.df['reading_score'] + self.df['writing_score']
        return self.df['total_score']

    def create_gpa(self):

        def update(grade):
            percetage = grade / 300 * 100
            
            if percetage > 90:
                return 'A'
            if percetage > 70:
                return 'B'
            if percetage > 60:
                return 'C'
            if percetage > 50:
                return 'D'
            return 'F'
        
        self.df['gpa'] = self.df['total_score'].apply(update)
        
        
        return self.df['gpa']

    
    
    # in this function we will do the following
    '''
        at the first, the idea of filling nan values is to assign them by the priority for the smallest group
        so:
        1) i will creat a dictionary contains group name and its frequency
        2) i will replace nan values with '-'
        3) i will update them by iteating over the dictionary and get the group that has smallest number of students
        4) i will increace this group by 1, and after that return the group name
        5) i will replace the groups name with Names of continents
    '''    
    def edit_groups(self):

        groups = self.df[self.df['group'].isna() == 0]['group'].value_counts()

        mappe = {}
        for index in groups.index:
            mappe[index] = groups[index]

        def update(group):

            if group == '-':
                mn  = 1e9
                name_of_group = ''
                
                for index in mappe:

                    if mappe[index] < mn:
                        mn = mappe[index]
                        name_of_group = index

                mappe[name_of_group] += 1
                return name_of_group
            
            return group

        
        self.df['group'] = self.df['group'].fillna('-')        
        self.df['group'] = self.df['group'].apply(update)


        group_mapping = {
            'group A': 'Europeans',
            'group B': 'Africans',
            'group C': 'Asians',
            'group D': 'Americans',
            'group E': 'Australians'

        }
        
        self.df['group'] = self.df['group'].replace(group_mapping)
        
        return self.df['group']


    # this function will repair parent_education

    '''
        my idea is to determine the parent education by the score of student
        so if it high, that means that his parent also had high degree
        after updating nan values i will replace some from some high school and some college
    '''
    def edit_parent_education(self):

        def update(i):
            if self.df.loc[i, 'total_score'] > 280:
                return 'masters degree'
            if self.df.loc[i, 'total_score'] > 250:
                return 'bachelors degree'
            if self.df.loc[i, 'total_score'] > 200:
                return 'associates degree'
            if self.df.loc[i, 'total_score'] > 150:
                return 'college'
            return 'high school'
        
        self.df['parent_education'] = self.df['parent_education'].fillna('-')        
        
        for i in range(len(self.df)):
            if self.df.loc[i,'parent_education'] == '-':
                self.df.loc[i,'parent_education'] = update(i)
        
        
        self.df['parent_education'] = self.df['parent_education'].str.replace('some high school', 'high school')
        self.df['parent_education'] = self.df['parent_education'].str.replace('some college', 'college')

        return self.df['parent_education']



    # filling nan values in test_preparation by not_detected
    def edit_test_preparation(self):
        
        self.df['test_preparation'] = self.df['test_preparation'].fillna('not_detected')
        self.df['test_preparation'] = self.df['test_preparation'].replace('none','No')

        return self.df['test_preparation']


    # filling nan values in parent_marital_status by not_detected
    def edit_parent_marital_status(self):
        
        self.df['parent_marital_status'] = self.df['parent_marital_status'].fillna('not_detected')

        return self.df['parent_marital_status']


    # filling nan values in practice_sport by not_detected
    def edit_practice_sport(self):
        
        self.df['practice_sport'] = self.df['practice_sport'].fillna('not_detected')

        return self.df['practice_sport']


    '''
     here i will repair the first_child column
     which in my idea i will considered that if child has high total_score
     that means that he is the first studen
    '''
    def edit_first_child(self):
        
        self.df['first_child'] = self.df['first_child'].fillna('-')

        def update(i):
            if self.df.loc[i, 'total_score'] > 250:
                return 'yes'
            return 'no'

        for i in range(len(self.df)):
            if self.df.loc[i,'first_child'] == '-':
                self.df.loc[i,'first_child'] = update(i)


        return self.df['first_child']


    # filling number_of_siblings with mean
    def edit_number_of_siblings(self):
        
        mean = int(self.df['number_of_siblings'].mean())
        
        self.df['number_of_siblings'] = self.df['number_of_siblings'].fillna(mean)

        return self.df['number_of_siblings']


    # filling transportation with school_bus
    def edit_transportation(self):
        
        
        self.df['transportation'] = self.df['transportation'].fillna('school_bus')

        return self.df['transportation']


    # filling week_study_hours
    '''
        it will based on the total_score, so if he had high total_score that means that 
        student spend more hours in studing 
    '''
    def edit_week_study_hours(self):
        
        self.df['week_study_hours'] = self.df['week_study_hours'].fillna('-')

        def update(i):
            if self.df.loc[i, 'total_score'] > 280:
                return '> 10'
            if self.df.loc[i, 'total_score'] > 200:
                return '5 - 10'
            return '< 5'



        for i in range(len(self.df)):
            if self.df.loc[i,'week_study_hours'] == '-':
                self.df.loc[i,'week_study_hours'] = update(i)


        return self.df['week_study_hours']



DC = Data_Cleaning()

In [57]:
temp_expanded_df = DC.remove()
temp_expanded_df = DC.rename_columns()
temp_expanded_df['total_socre'] = DC.create_total_score()
temp_expanded_df['gpa'] = DC.create_gpa()
temp_expanded_df['group'] = DC.edit_groups()
temp_expanded_df['parent_education'] = DC.edit_parent_education()
temp_expanded_df['test_preparation'] = DC.edit_test_preparation()
temp_expanded_df['parent_marital_status'] = DC.edit_parent_marital_status()
temp_expanded_df['practice_sport'] = DC.edit_practice_sport()
temp_expanded_df['first_child'] = DC.edit_first_child()
temp_expanded_df['number_of_siblings'] = DC.edit_number_of_siblings()
temp_expanded_df['transportation'] = DC.edit_transportation()
temp_expanded_df['week_study_hours'] = DC.edit_week_study_hours()

In [58]:
temp_expanded_df.isnull().sum()

gender                   0
group                    0
parent_education         0
lunch_type               0
test_preparation         0
parent_marital_status    0
practice_sport           0
first_child              0
number_of_siblings       0
transportation           0
week_study_hours         0
math_score               0
reading_score            0
writing_score            0
total_score              0
total_socre              0
gpa                      0
dtype: int64

In [59]:
temp_expanded_df.drop(columns='total_score', inplace=True)

In [60]:
temp_expanded_df

,gender,group,parent_education,lunch_type,test_preparation,parent_marital_status,practice_sport,first_child,number_of_siblings,transportation,week_study_hours,math_score,reading_score,writing_score,total_socre,gpa
0,female,Europeans,bachelor's degree,standard,No,married,regularly,yes,3.0,school_bus,< 5,71,71,74,216,B
1,female,Asians,college,standard,not_detected,married,sometimes,yes,0.0,school_bus,5 - 10,69,90,88,247,B
2,female,Africans,master's degree,standard,No,single,sometimes,yes,4.0,school_bus,< 5,87,93,91,271,A
3,male,Europeans,associate's degree,free/reduced,No,married,never,no,1.0,school_bus,5 - 10,45,56,42,143,F
4,male,Asians,college,standard,No,married,sometimes,yes,0.0,school_bus,5 - 10,76,78,75,229,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30636,female,Americans,high school,standard,No,single,sometimes,no,2.0,school_bus,5 - 10,59,61,65,185,C
30637,male,Australians,high school,standard,No,single,regularly,no,1.0,private,5 - 10,58,53,51,162,D
30638,female,Europeans,high school,free/reduced,completed,married,sometimes,no,1.0,private,5 - 10,61,70,67,198,C
30639,female,Americans,associate's degree,standard,completed,married,regularly,no,3.0,school_bus,5 - 10,82,90,93,265,B


In [61]:
%store temp_expanded_df

Stored 'temp_expanded_df' (DataFrame)


/home/ahmed/.local/lib/python3.10/site-packages/IPython/extensions/storemagic.py:229: UserWarning: using autorestore/temp_expanded_df requires you to install the `pickleshare` library.
  db[ 'autorestore/' + arg ] = obj
